In [7]:
!pip install matplotlib
!pip install seaborn


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 1.5 MB/s eta 0:00:000:00:010:00:01:01

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [75]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('jakarta_street_coordinate.csv')

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df = df[ ~df['coordinate'].isna() ].reset_index(drop=True)

In [5]:
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG JATI INDAH,"[-6.2599631, 106.773803]"
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG MASRIKI,"[-6.2626431, 106.7741125]"
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,H SAIMAN,"[-6.2614855, 106.7741336]"
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JATI INDAH,"[-6.2599631, 106.773803]"
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL BANK EXIM,"[-6.263339, 106.7745935]"


In [6]:
len(df)

25956

We know that when we searched for the coordinate for the roads, we missed a lot of rows. However, we need those coordinates and we cannot afford any rows with missing data and thus removed them.

Let's do gap analysis for the _kelurahan_. That is, by checking how many _kelurahan_ have little or close to zero rows after we dropped all rows that weren't geocoded by Nominatim

In [7]:
df_all = pd.read_csv('NJOP2021.csv')
df_all = df_all.drop('Unnamed: 0', axis=1)
len(df_all)

219439

In [8]:
df_all = df_all.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()
df_all.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 001 - PONDOK PINANG,1382
1,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 002 - KEBAYORAN LAMA SLT,633
2,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 003 - KEBAYORAN LAMA UTR,725
3,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 004 - C I P U L I R,652
4,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 005 - GROGOL SELATAN,1141


In [9]:
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df_all[col] = df_all[col].apply(lambda x: str(x).split(' - ')[1])
df_all.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,1382
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SLT,633
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA UTR,725
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,C I P U L I R,652
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,GROGOL SELATAN,1141


In [10]:
df_agg = df.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()

In [11]:
df_merged = df_all.merge(df_agg, on=['provinsi', 'kota', 'kecamatan', 'kelurahan'], how='outer', suffixes=('_all', '_coord'))
df_merged.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan_all,nama_jalan_coord
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,1382,329.0
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SLT,633,NaN
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA UTR,725,NaN
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,C I P U L I R,652,NaN
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,GROGOL SELATAN,1141,221.0


In [12]:
df_merged['nama_jalan_coord'] = df_merged['nama_jalan_coord'].fillna(0)

In [13]:
df_merged['pct_coord'] = df_merged['nama_jalan_coord']/df_merged['nama_jalan_all']*100

In [14]:
df_merged.sort_values('pct_coord').head(40)

,provinsi,kota,kecamatan,kelurahan,nama_jalan_all,nama_jalan_coord,pct_coord
42,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN BARU,P U L O,224,0.0,0.000000
38,DKI JAKARTA,JAKARTA SELATAN,P A N C O R A N,C I K O K O,121,0.0,0.000000
39,DKI JAKARTA,JAKARTA SELATAN,P A N C O R A N,P A N C O R A N,275,0.0,0.000000
17,DKI JAKARTA,JAKARTA SELATAN,PASAR MINGGU,R A G U N A N,375,0.0,0.000000
47,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN BARU,S E L O N G,93,0.0,0.000000
49,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN BARU,S E N A Y A N,117,0.0,0.000000
53,DKI JAKARTA,JAKARTA SELATAN,SETIA BUDI,K A R E T,190,0.0,0.000000
56,DKI JAKARTA,JAKARTA SELATAN,SETIA BUDI,G U N T U R,260,0.0,0.000000
34,DKI JAKARTA,JAKARTA SELATAN,P A N C O R A N,KALI BATA,313,0.0,0.000000
29,DKI JAKARTA,JAKARTA SELATAN,MAMPANG PRAPATAN,B A N G K A,259,0.0,0.000000


We can see that some _kelurahan_ didn't even get any coordinate because of four main reasons:
1. `P U L O` should be written as `PULO`, the same problem goes until `C I P U L I R`
2. `UTR` should be written as `Utara` and `SLT` as `SELATAN`
3. `PAL MERIAM`, `KALI ANYAR`, `RAWA JATI`, `SETIA BUDI` should be written without space
4. `BALIMESTER` should be written as `BALI MESTER`

We'll focus on repairing those critical weak points using our geocode

In [15]:
reparse = df_merged.sort_values('pct_coord').head(21)['kelurahan'].to_list()

In [16]:
reparse.remove('HARAPAN MULIA')
reparse.remove('BIDARA CINA')

In [17]:
df_patch = pd.read_csv('./NJOP2021.csv')
df_patch = df_patch[['provinsi', 'kota', 'kecamatan', 'kelurahan', 'nama_jalan']]
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df_patch[col] = df_patch[col].apply(lambda x: str(x).split(' - ')[1])
df_patch = df_patch.drop_duplicates(ignore_index=True)
df_patch.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG FLAMBOYAN
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H MASRIKI
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAEMIN
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAIMIN
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H. SAIMIN


In [18]:
df_patch = df_patch[ df_patch['kelurahan'].isin(reparse) ]

In [19]:
remove_spaces = ['P U L O', 'C I K O K O', 'P A N C O R A N', 'R A G U N A N', 'S E L O N G', 'S E N A Y A N', 'K A R E T', 'G U N T U R', 'KALI BATA', 'B A N G K A', 'U L U J A M I', 'C I P U L I R', 'PAL MERIAM', 'KALI ANYAR', 'RAWA JATI', 'SETIA BUDI']
df_patch['kelurahan'] = df_patch['kelurahan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df_patch['kelurahan'] = df_patch['kelurahan'].str.replace('KEBAYORAN LAMA UTR', 'KEBAYORAN LAMA UTARA').str.replace('KEBAYORAN LAMA SLT', 'KEBAYORAN LAMA SELATAN').str.replace('BALIMESTER', 'BALI MESTER')

In [20]:
df_patch = df_patch.drop_duplicates(ignore_index=True)
df_patch.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,BUNGUR RAYA
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,GG H MI'AH
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,JL ALTERI
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,JL ARTERI
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,JL ARTERI PONDOK PINANG


In [21]:
from geopy.geocoders import Nominatim # https://nominatim.org/
import time

In [22]:
dict = {}
for idx in range(len(df_patch)):
    print('Now on index', idx)
    try:
        geolocator = Nominatim(user_agent="yusuf-application")
        location = geolocator.geocode(df_patch.iloc[idx]['nama_jalan']+', '+df_patch.iloc[idx]['kelurahan'], timeout=10)
        if location:
            dict[idx] = [location.latitude, location.longitude]
    except:
        idx -= 1
        time.sleep(60) # wait for internet to reconnect

Now on index 0
Now on index 1
Now on index 2
Now on index 3
Now on index 4
Now on index 5
Now on index 6
Now on index 7
Now on index 8
Now on index 9
Now on index 10
Now on index 11
Now on index 12
Now on index 13
Now on index 14
Now on index 15
Now on index 16
Now on index 17
Now on index 18
Now on index 19
Now on index 20
Now on index 21
Now on index 22
Now on index 23
Now on index 24
Now on index 25
Now on index 26
Now on index 27
Now on index 28
Now on index 29
Now on index 30
Now on index 31
Now on index 32
Now on index 33
Now on index 34
Now on index 35
Now on index 36
Now on index 37
Now on index 38
Now on index 39
Now on index 40
Now on index 41
Now on index 42
Now on index 43
Now on index 44
Now on index 45
Now on index 46
Now on index 47
Now on index 48
Now on index 49
Now on index 50
Now on index 51
Now on index 52
Now on index 53
Now on index 54
Now on index 55
Now on index 56
Now on index 57
Now on index 58
Now on index 59
Now on index 60
Now on index 61
Now on index 62
No

In [23]:
df_patch['coordinate'] = None
for i in dict:
    df_patch['coordinate'].iloc[i] = dict[i]

In [24]:
df_patch = df_patch[ ~df_patch['coordinate'].isna() ]

In [25]:
df_patch['kelurahan'].value_counts()

kelurahan
KEBAYORAN LAMA UTARA      214
ULUJAMI                   173
CIPULIR                   145
KEBAYORAN LAMA SELATAN    137
KALIBATA                  113
BANGKA                    109
PANCORAN                  108
RAGUNAN                    93
PULO                       85
PALMERIAM                  77
RAWAJATI                   67
SELONG                     64
BALI MESTER                48
CIKOKO                     46
KARET                      46
SENAYAN                    40
GUNTUR                     29
SETIABUDI                  21
KALIANYAR                   2
Name: count, dtype: int64

The result is much better now

In [26]:
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG JATI INDAH,"[-6.2599631, 106.773803]"
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG MASRIKI,"[-6.2626431, 106.7741125]"
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,H SAIMAN,"[-6.2614855, 106.7741336]"
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JATI INDAH,"[-6.2599631, 106.773803]"
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL BANK EXIM,"[-6.263339, 106.7745935]"


In [27]:
df_total = pd.concat([df, df_patch], axis=0)

In [28]:
df_total = df_total.reset_index(drop=True)
df_total = df_total.loc[df_total.astype(str).drop_duplicates().index]
df_total = df_total.reset_index(drop=True)

In [29]:
df_total.to_csv('jakarta_street_coordinate_patched.csv', index=False)

In [32]:
df_agg = df_total.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()

In [33]:
df_all = pd.read_csv('NJOP2021.csv')
df_all = df_all.drop('Unnamed: 0', axis=1)
df_all = df_all.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df_all[col] = df_all[col].apply(lambda x: str(x).split(' - ')[1])
remove_spaces = ['P U L O', 'C I K O K O', 'P A N C O R A N', 'R A G U N A N', 'S E L O N G', 'S E N A Y A N', 'K A R E T', 'G U N T U R', 'KALI BATA', 'B A N G K A', 'U L U J A M I', 'C I P U L I R', 'PAL MERIAM', 'KALI ANYAR', 'RAWA JATI', 'SETIA BUDI']
df_all['kelurahan'] = df_all['kelurahan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df_all['kelurahan'] = df_all['kelurahan'].str.replace('KEBAYORAN LAMA UTR', 'KEBAYORAN LAMA UTARA').str.replace('KEBAYORAN LAMA SLT', 'KEBAYORAN LAMA SELATAN').str.replace('BALIMESTER', 'BALI MESTER')

In [36]:
df_merged = df_all.merge(df_agg, on=['provinsi', 'kota', 'kecamatan', 'kelurahan'], how='outer', suffixes=('_all', '_coord'))
df_merged['nama_jalan_coord'] = df_merged['nama_jalan_coord'].fillna(0)
df_merged['pct_coord'] = df_merged['nama_jalan_coord']/df_merged['nama_jalan_all']*100
df_merged.sort_values('pct_coord').head(40)


,provinsi,kota,kecamatan,kelurahan,nama_jalan_all,nama_jalan_coord,pct_coord
155,DKI JAKARTA,JAKARTA PUSAT,KEMAYORAN,HARAPAN MULIA,125.0,1,0.800000
211,DKI JAKARTA,JAKARTA BARAT,TAMBORA,KALIANYAR,116.0,2,1.724138
95,DKI JAKARTA,JAKARTA TIMUR,JATINEGARA,BIDARA CINA,231.0,4,1.731602
263,DKI JAKARTA,KEPULAUAN SERIBU,KEPULAUAN SERIBU UTARA,PULAU KELAPA,42.0,1,2.380952
236,DKI JAKARTA,JAKARTA UTARA,PADEMANGAN,PADEMANGAN BARAT,354.0,9,2.542373
218,DKI JAKARTA,JAKARTA BARAT,TAMBORA,JEMBATAN LIMA,232.0,8,3.448276
108,DKI JAKARTA,JAKARTA TIMUR,DUREN SAWIT,MALAKA SARI,749.0,33,4.405874
261,DKI JAKARTA,KEPULAUAN SERIBU,KEPULAUAN SERIBU UTARA,PULAU HARAPAN,44.0,2,4.545455
136,DKI JAKARTA,JAKARTA PUSAT,TANAH ABANG,KAMPUNG BALI,191.0,9,4.712042
259,DKI JAKARTA,JAKARTA UTARA,CILINCING,SEMPER BARAT,749.0,44,5.874499


In [40]:
df_merged['pct_coord'].describe()

count    267.000000
mean      25.038842
std       11.641125
min        0.800000
25%       17.750112
50%       23.701299
75%       31.083828
max       71.428571
Name: pct_coord, dtype: float64

## Spatial Visualization

In [41]:
!pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 588.4 kB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def generateBaseMap(default_location=[-6.22, 106.83], default_zoom_start=11):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

In [3]:
import folium
basemap=generateBaseMap()

Let's plot the mean of latitude and longitude location of the _Kelurahan_ with the number of parsed streets

In [4]:
df = pd.read_csv('jakarta_street_coordinate_patched.csv')
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG JATI INDAH,"[-6.2599631, 106.773803]"
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG MASRIKI,"[-6.2626431, 106.7741125]"
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,H SAIMAN,"[-6.2614855, 106.7741336]"
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JATI INDAH,"[-6.2599631, 106.773803]"
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL BANK EXIM,"[-6.263339, 106.7745935]"


In [5]:
df['latitude'] = df['coordinate'].apply(lambda x: re.search(r'\[([-\d\.]+),', x)[1]).astype('float')
df['longitude'] = df['coordinate'].apply(lambda x: re.search(r',\s([-\d\.]+)]$', x)[1]).astype('float')

In [6]:
cols = ['kelurahan', 'nama_jalan', 'latitude', 'longitude']
df_map = df[cols].groupby('kelurahan').aggregate({'latitude':'median', 'longitude':'median', 'nama_jalan':'nunique'}).reset_index()

In [7]:
HeatMap(df_map[['latitude','longitude','nama_jalan']]).add_to(basemap)
# basemap

We can see that almost all of Jakarta is plotted with the heat map

In [8]:
FastMarkerCluster(data=df_map[['latitude','longitude','nama_jalan']].values.tolist()).add_to(basemap)

In [9]:
basemap # you can try to zoom in!

Let's try to plot every parsed street coordinate and the NJOP price now!

In [10]:
df_njop = pd.read_csv('NJOP2021.csv')
njop_cols = ['kelurahan', 'nama_jalan', 'ave_njb']
df_njop = df_njop[njop_cols]
df_njop['kelurahan'] = df_njop['kelurahan'].apply(lambda x: str(x).split(' - ')[1])

In [11]:
df_njop.head()

,kelurahan,nama_jalan,ave_njb
0,PONDOK PINANG,GG FLAMBOYAN,"5,763,000"
1,PONDOK PINANG,GG FLAMBOYAN,"5,223,000"
2,PONDOK PINANG,GG H MASRIKI,"5,763,000"
3,PONDOK PINANG,GG H SAEMIN,"5,763,000"
4,PONDOK PINANG,GG H SAEMIN,"5,223,000"


In [12]:
df_map = df[cols].merge(df_njop, how='inner', on=['kelurahan', 'nama_jalan'])
df_map['ave_njb'] = df_map['ave_njb'].str.replace(',', '').astype('float')
df_map = df_map.groupby(['kelurahan', 'nama_jalan']).aggregate({'latitude': 'median', 'longitude':'median', 'ave_njb':'mean'})

In [13]:
df_map

latitude   longitude     ave_njb
kelurahan     nama_jalan                                             
ANCOL         ANCOL BARAT I         -6.118674  106.820483   7623000.0
              DUNIA FANTASI         -6.124209  106.832142  26343000.0
              GG GLORIA             -6.130293  106.821545   7455000.0
              GG LODAN RAYA         -6.131199  106.823285  14105000.0
              GG PARANG TRITIS RAYA -6.125886  106.821410   7455000.0
...                                       ...         ...         ...
WIJAYA KUSUMA PERDANA II            -6.155730  106.773258  11305000.0
              TAMAN MUTIARA         -6.159929  106.771974  11305000.0
              UTAMA SAKTI IV        -6.151969  106.778334  11305000.0
              UTAMA SAKTI VIII      -6.153271  106.777934  11305000.0
              WIJAYA KUSUMA IV      -6.153584  106.771412   8145000.0

[25956 rows x 3 columns]

In [14]:
basemap=generateBaseMap()
HeatMap(df_map[['latitude','longitude','ave_njb']], radius=5).add_to(basemap)
FastMarkerCluster(data=df_map[['latitude','longitude','ave_njb']].values.tolist()).add_to(basemap)
basemap

Let's take some time to celebrate because we have already combined NJOP data from Governmental PDF and its coordinate from Nominatim (OpenStreetMap). We are fully aware that our data source is free and thus quite limited.

However, the idea is still applicable and we use Google Maps API for more serious projects.

We only parsed a few data, but we must remember that spatial data is correlated by spatial correlation. Therefore, this information will probably still be useful to improve our model.

Next, we are going to define a function that takes a coordinate to generate average NJOP value near the facilities

In [15]:
# let's save the the df_map
df_map.reset_index().to_csv('NJOP_coordinate.csv', index=False)

In [42]:
import geopy.distance

In [80]:
# https://stackoverflow.com/questions/1253499/simple-calculations-for-working-with-lat-lon-and-km-distance
# Latitude: 1 deg = 110.574 km
# Longitude: 1 deg = 111.320*cos(latitude) km
def get_average_njop_near(lat, long, radius_km, df_map):
    """
    Dependencies: numpy as np, pandas as pd, geopy.distance
    Input: Latitude (float), Longitude (float), Radius in km (float)
    Output: Average NJOP value based on df_map in the radius (float)
    """
    d_lat = radius_km/110 # let's use upper bound of 110000
    d_long = radius_km/111 # let's assume upper bound of cos(0) and 111000
    df = df_map.copy(deep=True)
    df = df[ (df['latitude'].between(lat-d_lat, lat+d_lat)) & (df['longitude'].between(long-d_long, long+d_long)) ] # bbox
    try:
        df['distance'] = df.apply(lambda x: geopy.distance.distance((lat, long) , (x['latitude'], x['longitude'])).km, axis=1)
    except:
        df['distance'] = np.nan
    df = df[ df['distance'] <= radius_km ]
    return [df['ave_njb'].mean(), df['ave_njb'].count(), df['ave_njb'].min(), df['ave_njb'].max(), df['ave_njb'].std()]

Now, determining the radius was quite an arbitrary job. Let's do a random sampling of 50 addresses from Lamudi houses to check whether we can use 1 km

In [56]:
df_lamudi = pd.concat([pd.read_csv('lamudi_house_dataset.csv'), pd.read_csv('lamudi_house_dataset_2.csv')], axis=0)
df_lamudi = df_lamudi.drop('Unnamed: 0', axis=1) # from the creation of csv
df_lamudi.head()

,data_price_s,data_category_s,data_subcategories_s,data_bedrooms_s,data_bathrooms_s,data_building_size_s,data_land_size_s,data_furnished_s,data_sku_s,data_geo_point_s,page_link_s,parent_link_s
0,2.000000e+08,house,"[""house"",""single-family-house""]",2,2.0,52.0,40.0,NaN,HO63342840B9EB3ID,NaN,https://www.lamudi.co.id/wujudkan-dan-desain-h...,https://www.lamudi.co.id/jakarta/kepulauan-ser...
1,1.250000e+09,house,"[""house"",""single-family-house""]",3,2.0,200.0,200.0,NaN,HO63CE42B5E3B0AID,NaN,https://www.lamudi.co.id/rumah-kedungmundu-mur...,https://www.lamudi.co.id/jakarta/kepulauan-ser...
2,6.910000e+08,house,"[""house"",""single-family-house""]",2,2.0,55.0,60.0,NaN,HO62B162B0CB941ID,NaN,https://www.lamudi.co.id/rumah-2-lt-hot-promo-...,https://www.lamudi.co.id/jakarta/kepulauan-ser...
3,1.200000e+09,house,"[""house"",""single-family-house""]",3,1.0,80.0,150.0,NaN,HO5E2567A098B1DID,"[106.95499,-6.19651]",https://www.lamudi.co.id/rumah-siap-huni-luas-...,https://www.lamudi.co.id/jakarta/kepulauan-ser...
4,2.100000e+09,house,"[""house"",""single-family-house""]",6,2.0,150.0,305.0,NaN,HO60111E9AC8A8EID,"[106.95499,-6.19651]",https://www.lamudi.co.id/rumah-siap-huni-luas-...,https://www.lamudi.co.id/jakarta/kepulauan-ser...


In [57]:
import random

In [58]:
random.seed(42)
addresses = df_lamudi[ ~df_lamudi['data_geo_point_s'].isna() ]['data_geo_point_s'].unique()
addresses = random.sample(sorted(addresses), 50)

In [59]:
dict_check = {'mean_1_km': [None]*50, 'mean_2_km': [None]*50, 'count_1_km': [None]*50, 'count_2_km': [None]*50}
for i in range(len(addresses)):
    address = eval(addresses[i])
    for j in [1, 2]:
        sol = get_average_njop_near(address[1], address[0], j, df_map)
        dict_check[f'mean_{j}_km'][i] = sol[0]
        dict_check[f'count_{j}_km'][i] = sol[1]

In [60]:
for names in dict_check:
    print(pd.DataFrame(data={names: dict_check[names]}).describe())

          mean_1_km
count  4.100000e+01
mean   1.047045e+07
std    7.058057e+06
min    2.993551e+06
25%    4.454271e+06
50%    8.143599e+06
75%    1.536497e+07
max    2.522670e+07
          mean_2_km
count  4.200000e+01
mean   9.549830e+06
std    5.456866e+06
min    2.977584e+06
25%    4.982829e+06
50%    8.499530e+06
75%    1.493648e+07
max    2.032567e+07
       count_1_km
count   50.000000
mean   109.520000
std     85.876333
min      0.000000
25%     46.000000
50%     91.500000
75%    168.000000
max    287.000000
        count_2_km
count    50.000000
mean    393.640000
std     276.722491
min       0.000000
25%     193.000000
50%     399.500000
75%     615.750000
max    1002.000000


We can see that there are coordinates with no parsed coordinates within 1 and 2 km. Let's check those coordinates

In [61]:
null_neighbors = []
for i in range(len(addresses)):
    if dict_check['mean_1_km'][i] != dict_check['mean_1_km'][i]: # check if nan
        null_neighbors.append([eval(addresses[i])[1], eval(addresses[i])[0]])

In [62]:
basemap=generateBaseMap()
HeatMap(df_map[['latitude','longitude','ave_njb']], radius=5).add_to(basemap)
for coord in null_neighbors:
    folium.Marker(location=coord).add_to(basemap)
basemap

From the heatmap, we can see that the markers were outside of Jakarta. Therefore, let's just use the 1 km since it has more variability

In [81]:
geo_point_s = sorted(df_lamudi['data_geo_point_s'].astype('str').unique())
print('Addresses count:', len(geo_point_s))
average_njop = []
neighbors = []
counter = 0
min_ave_njop = []
max_ave_njop = []
std_ave_njop = []
for address in geo_point_s:
    counter += 1
    print('Now on address:', counter)
    try:
        address = eval(address)
        sol = get_average_njop_near(address[1], address[0], 1, df_map)
        average_njop.append(sol[0])
        neighbors.append(sol[1])
        min_ave_njop.append(sol[2])
        max_ave_njop.append(sol[3])
        std_ave_njop.append(sol[4])
    except:
        average_njop.append(np.nan)
        neighbors.append(np.nan)
        min_ave_njop.append(np.nan)
        max_ave_njop.append(np.nan)
        std_ave_njop.append(np.nan)

Addresses count: 3449
Now on address: 1
Now on address: 2
Now on address: 3
Now on address: 4
Now on address: 5
Now on address: 6
Now on address: 7
Now on address: 8
Now on address: 9
Now on address: 10
Now on address: 11
Now on address: 12
Now on address: 13
Now on address: 14
Now on address: 15
Now on address: 16
Now on address: 17
Now on address: 18
Now on address: 19
Now on address: 20
Now on address: 21
Now on address: 22
Now on address: 23
Now on address: 24
Now on address: 25
Now on address: 26
Now on address: 27
Now on address: 28
Now on address: 29
Now on address: 30
Now on address: 31
Now on address: 32
Now on address: 33
Now on address: 34
Now on address: 35
Now on address: 36
Now on address: 37
Now on address: 38
Now on address: 39
Now on address: 40
Now on address: 41
Now on address: 42
Now on address: 43
Now on address: 44
Now on address: 45
Now on address: 46
Now on address: 47
Now on address: 48
Now on address: 49
Now on address: 50
Now on address: 51
Now on address: 52

In [82]:
len(sorted(set(average_njop)))

2732

In [86]:
df_lamudi_coord_njop = pd.DataFrame(data={'data_geo_point_s': geo_point_s, 'njop':average_njop, 'neighbors':neighbors
                                          , 'min_njop': min_ave_njop, 'max_njop': max_ave_njop, 'std_njop': std_ave_njop})

In [87]:
df_lamudi_coord_njop

,data_geo_point_s,njop,neighbors,min_njop,max_njop,std_njop
0,"[0,0]",NaN,0.0,NaN,NaN,NaN
1,"[1,-6.1136504]",NaN,0.0,NaN,NaN,NaN
2,"[1,-6.1368875]",NaN,0.0,NaN,NaN,NaN
3,"[1,-6.2317563]",NaN,0.0,NaN,NaN,NaN
4,"[1,-6.3454]",NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...
3444,"[40,12]",NaN,0.0,NaN,NaN,NaN
3445,"[50,13]",NaN,0.0,NaN,NaN,NaN
3446,"[89.8765,-56.351]",NaN,0.0,NaN,NaN,NaN
3447,"[9,-6.1886]",NaN,0.0,NaN,NaN,NaN


In [89]:
df_lamudi['data_geo_point_s'] = df_lamudi['data_geo_point_s'].astype('str')
df_lamudi = df_lamudi.merge(df_lamudi_coord_njop, on='data_geo_point_s', how='left')
df_lamudi

,data_price_s,data_category_s,data_subcategories_s,data_bedrooms_s,data_bathrooms_s,data_building_size_s,data_land_size_s,data_furnished_s,data_sku_s,data_geo_point_s,page_link_s,parent_link_s,njop,neighbors,min_njop,max_njop,std_njop
0,2.000000e+08,house,"[""house"",""single-family-house""]",2,2.0,52.0,40.0,NaN,HO63342840B9EB3ID,nan,https://www.lamudi.co.id/wujudkan-dan-desain-h...,https://www.lamudi.co.id/jakarta/kepulauan-ser...,NaN,NaN,NaN,NaN,NaN
1,1.250000e+09,house,"[""house"",""single-family-house""]",3,2.0,200.0,200.0,NaN,HO63CE42B5E3B0AID,nan,https://www.lamudi.co.id/rumah-kedungmundu-mur...,https://www.lamudi.co.id/jakarta/kepulauan-ser...,NaN,NaN,NaN,NaN,NaN
2,6.910000e+08,house,"[""house"",""single-family-house""]",2,2.0,55.0,60.0,NaN,HO62B162B0CB941ID,nan,https://www.lamudi.co.id/rumah-2-lt-hot-promo-...,https://www.lamudi.co.id/jakarta/kepulauan-ser...,NaN,NaN,NaN,NaN,NaN
3,1.200000e+09,house,"[""house"",""single-family-house""]",3,1.0,80.0,150.0,NaN,HO5E2567A098B1DID,"[106.95499,-6.19651]",https://www.lamudi.co.id/rumah-siap-huni-luas-...,https://www.lamudi.co.id/jakarta/kepulauan-ser...,5.311486e+06,182.0,2640000.0,8145000.0,2.108699e+06
4,2.100000e+09,house,"[""house"",""single-family-house""]",6,2.0,150.0,305.0,NaN,HO60111E9AC8A8EID,"[106.95499,-6.19651]",https://www.lamudi.co.id/rumah-siap-huni-luas-...,https://www.lamudi.co.id/jakarta/kepulauan-ser...,5.311486e+06,182.0,2640000.0,8145000.0,2.108699e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79135,4.300000e+09,house,"[""house"",""single-family-house""]",3,2.0,245.0,126.0,NaN,HO6364C8B7A3C09ID,"[106.865237,-6.137171]",https://www.lamudi.co.id/rumah-sunter-agung-mu...,https://www.lamudi.co.id/jakarta/jakarta-utara...,1.916743e+07,153.0,4155000.0,28855000.0,3.435436e+06
79136,2.900000e+10,house,"[""house"",""single-family-house""]",6,4.0,1200.0,1040.0,NaN,HO63D662483CD7CID,"[106.865237,-6.137171]",https://www.lamudi.co.id/rumah-sunter-agung-in...,https://www.lamudi.co.id/jakarta/jakarta-utara...,1.916743e+07,153.0,4155000.0,28855000.0,3.435436e+06
79137,3.300000e+09,house,"[""house"",""single-family-house""]",3,3.0,200.0,96.0,NaN,HO643FB884DFB95ID,"[106.865237,-6.137171]",https://www.lamudi.co.id/rumah-minimalis-3-lan...,https://www.lamudi.co.id/jakarta/jakarta-utara...,1.916743e+07,153.0,4155000.0,28855000.0,3.435436e+06
79138,2.500000e+09,house,"[""house"",""single-family-house""]",4,3.0,110.0,93.0,NaN,HO642FE87DB55C9ID,"[106.865237,-6.137171]",https://www.lamudi.co.id/dijual-rumah-sunter-a...,https://www.lamudi.co.id/jakarta/jakarta-utara...,1.916743e+07,153.0,4155000.0,28855000.0,3.435436e+06


Finally, we merged the information between Lamudi prices with NJOP data

In [90]:
df_lamudi.to_csv('lamudi_njop_dataset.csv', index=False)

### Self Doodle, Nominatim and OpenStreetMap Really Doesn't Have the Data in Their Database

Since there are only 27k streets with coordinate, and we have seen that there are some locations that weren't able to be geocoded because of weird abbreviation (e.g. _UTR_ for _UTARA_). I suspect we will find lots of those cases. Let's see whether this hypothesis is true

In [4]:
df = pd.read_csv('jakarta_street_coordinate.csv')
df = df.drop('Unnamed: 0', axis=1)

In [32]:
words_list = []
words_set = set({})
final_words_set = set({})
for name in df['nama_jalan'].to_list():
    for word in name.split(' '):
        words_list.append(word)
        words_set.add(word)
for word in words_set:
    if words_list.count(word) > 100: # to remove insignificant words
        final_words_set.add(word)

In [40]:
def check_word(df, word, col, target):
    df = df.copy(deep=True)
    df[word+'_check'] = df[col].apply(lambda x: word in x.split(' ')) # population are rows with the word
    df[target+'_check'] = df[target].apply(lambda x: False if x != x else True) # check nan in target column
    df = df[ df[word+'_check'] ]
    try: # there is probability no True value
        num_found = df[target+'_check'].value_counts()[True] 
    except:
        num_found = 0
    try: # there is probability no False value
        num_unfound = df[target+'_check'].value_counts()[False]
    except:
        num_unfound = 0
    return [num_found, num_unfound, ((num_found+1)*100)/(num_found+num_unfound+2)] # Laplace Smoothing to prevent error

In [41]:
check_word(df, 'GG.', 'nama_jalan', 'coordinate')

[56, 824, 6.462585034013605]

In [42]:
words_dict = {}
for word in final_words_set:
    words_dict[word] = check_word(df, word, 'nama_jalan', 'coordinate')

In [43]:
checked_words = pd.DataFrame.from_dict(words_dict, orient='index', columns=['found', 'unfound', 'pct_found'])

In [51]:
checked_words.sort_values('unfound', ascending=False).head(30)

,found,unfound,pct_found
JL,17763,71571,19.884481
GG,2862,22626,11.231856
I,1433,8970,13.781836
II,1954,7219,21.307902
RAYA,1644,6434,20.358911
H,85,5968,1.420314
KO,2,5808,0.051617
III,1528,4465,25.504587
KP,676,4143,14.042730
BLOK,50,4090,1.231289


There are many interesting information here:
1. _JL_ should be abbreviation for _Jalan_ (Street)
2. _GG_ should be abbreviation for _Gang_ (Alley)
3. I don't know what _KO_ is. Need to be checked
4. I don't know what _KP_ is. Need to be checked
5. _KAV_ should be abbreviation for _Kavling_ (Parcel of Land -- No real English translation, though)

In [52]:
def check_row_by_word(df, word):
    df = df.copy(deep=True)
    df['check_word'] = df['nama_jalan'].apply(lambda x: word in x.split(' '))
    return df[ df['check_word'] ]

In [ ]:
def print_geolocation_if_exist(street):
    geolocator = Nominatim(user_agent="yusuf-application")
    location = geolocator.geocode(street, timeout=10)
    if location:
        print([location.latitude, location.longitude])

Let's check whether using the non-abbreviated form improves the performance

In [80]:
check_row_by_word(df, 'BLK').head(20)

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate,check_word
77,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL P LEBAK LESTARI BLK,NaN,True
134,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL PINANG EMAS I BLK F-4 KO PU,NaN,True
202,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL PINANG PERAK III BLK V,NaN,True
472,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL ALAM SEGAR V BLK IV UW-15,NaN,True
473,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL ALAM SEGAR V BLK UW,NaN,True
508,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL ALAM SEGAR XI BLK IV,NaN,True
1083,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL KENCANA PERMAI II BLK SR 14,NaN,True
1136,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL SEKOLAH KENCANA BLK II,NaN,True
1258,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL DUTA INDAH III BLK TL PS-12,NaN,True
2143,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA UTR,JL TAMAN GANDARIA BLK E,NaN,True


In [ ]:
print_geolocation_if_exist('JL TAMAN GANDARIA, KEBAYORAN LAMA UTARA')

NameError: name 'print_geolocation_if_exist' is not defined